Hello, my name is Aaron Kann, and this is an analysis on soccer stats.  The goal of this first analysis is to determine whether,
in general, a players last 5 stats are a better predictor of the next match's output as compared to the season to date.

In [1]:
import sys

sys.path.append('..')  # Add parent folder to the module search path
import fbref_scrape as fbref
from constants import *


Firstly, the data for the premier league can be scraped through my code.  It will take a long time, so only uncomment the code if absolutely necessary.
The data should be stored in a folder called Premier_League_Matchlogs and accessed through there, so unless you are updating the folder, it is unnessecary to run the following code strip.

In [3]:
import json, csv, os
import pandas as pd
import numpy as np
from scipy.stats import poisson

# uncommenting the following line will scrape the premier league data
fbref.SoccerLeague(fbref.premier_league_24_url, {
    'list_teams': [16],
    'file_location': '..'
})
break

https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats
200.  We're in
https://fbref.com/en/squads/1df6b87e/2023-2024/Sheffield-United-Stats
200.  We're in
https://fbref.com/en/players/eebacc07/Gustavo-Hamer
200.  We're in
Coventry City
Sheffield United
/en/squads/f7e3dfe9/2022-2023/c10/Coventry-City-Stats-Championship
https://fbref.com/en/squads/1df6b87e/2023-2024/Sheffield-United-Stats
f7e3dfe9
1df6b87e
2. Championship
Premier League
/en/comps/10/2022-2023/2022-2023-Championship-Stats
https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats
10
9
False
False
Coventry City
Sheffield United
/en/squads/f7e3dfe9/2022-2023/c10/Coventry-City-Stats-Championship
https://fbref.com/en/squads/1df6b87e/2023-2024/Sheffield-United-Stats
f7e3dfe9
1df6b87e
2. Championship
Premier League
/en/comps/10/2022-2023/2022-2023-Championship-Stats
https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats
10
9
False
False
Coventry City
Sheffield United
/en/squads/f7e3dfe9/

SyntaxError: 'break' outside loop (3610227330.py, line 11)

Next, we need to take the indiviual player's gamelogs and create a giant database of games.  
Because the variables we are trying to measure aren't explicitly within the data, we need to calculate them.

In [ ]:
# read in the data
team_df = pd.read_csv('Premier_League_Team_Stats.csv')

data_folder = 'Premier_League_Player_Matchlogs'
data_files = os.listdir(data_folder)
dfs = []

for file in data_files:
    if file.endswith('.csv'):
        file_path = os.path.join(data_folder, file)
        # read the csv file and process the data
        df = pd.read_csv(file_path)
        # df = add_name_to_df(df, file)
        dfs.append(df)

dfs[0].head()


,game_started,game_count,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,...,sca,gca,passes_completed,passes_pct,progressive_passes,carries,progressive_carries,take_ons,take_ons_won,match_report
0,Y,1,0,90,0,0,0,0,0,2,...,3,0,22,81.5,3,20,3,2,0,Match Report
1,Y,2,0,89,0,0,1,0,0,1,...,1,0,21,80.8,2,18,2,2,2,Match Report
2,Y,3,0,90,0,0,6,0,0,1,...,3,0,26,83.9,4,30,2,3,1,Match Report
3,Y,4,0,90,0,0,1,0,0,4,...,3,1,37,78.7,3,31,3,1,1,Match Report
4,Y,5,0,79,0,0,4,0,0,0,...,3,0,11,64.7,1,16,2,2,1,Match Report


In [ ]:
# def add_name_to_df(df, filename):
#     for row in df

def update_avgs(season_avgs, row, index):
    #iterate through the stats in season_avgs and update the averages
    for stat in season_avgs.keys():
        if stat in row.keys():
            season_avgs[stat] = (season_avgs[stat]*index + row[stat])/(index+1)
    return season_avgs

def update_last_five_avgs(last_five_avgs, df, index):
    #iterate through the stats in last_five_avgs and update the averages
    for stat in last_five_avgs.keys():
        if stat in df.keys():
            last_five_avgs[stat] = (last_five_avgs[stat]*5 + df.iloc[index][stat] - df.iloc[index-5][stat])/(5)
    return last_five_avgs

In [ ]:
stats_wanted = fbref.get_stats_wanted('fbref')
stats_wanted.remove('name')
stats_wanted.remove('team')
stats_wanted.append('game_count')
df_as_list = list()

for i in range(len(dfs)): #iterates through each player's data
    
    # create a new row, with the season totals as of matchweek 0 (ie all zero)
    season_avgs = pd.Series(data=[0]*len(stats_wanted), index=stats_wanted)
    last_five_avgs = pd.Series(data=[0]*len(stats_wanted), index=stats_wanted)
    # print(season_avgs, last_five_avgs)

    for index, row in dfs[i].iterrows(): #iterates through each game of each players data
        dfrow = pd.concat([row, season_avgs.add_suffix("_szn"), last_five_avgs.add_suffix("_l5")], axis=0)

        #TODO: scale this to all stats
        dfrow['passes_l5_minus_szn'] = dfrow['passes_l5'] - dfrow['passes_szn']
        dfrow['opp_passes_against'] = team_df[team_df['name'] == row['opponent']]['passes_against'].values[0] / team_df['passes_against'].mean()


        season_avgs = update_avgs(season_avgs, row, index)
        if index >= 5:
            last_five_avgs = update_last_five_avgs(last_five_avgs, dfs[i], index)
        else:
            last_five_avgs = update_avgs(last_five_avgs, row, index)

        df_as_list.append(dfrow)

    #end game loop
#end player loop
        


df = pd.DataFrame(df_as_list)
df.head()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_16952\3311213170.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '89.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  season_avgs[stat] = (season_avgs[stat]*index + row[stat])/(index+1)
C:\Users\Aaron\AppData\Local\Temp\ipykernel_16952\3311213170.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '46.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  season_avgs[stat] = (season_avgs[stat]*index + row[stat])/(index+1)
C:\Users\Aaron\AppData\Local\Temp\ipykernel_16952\3311213170.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype f

,game_started,game_count,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,...,shots_on_target_l5,passes_l5,interceptions_l5,fouls_l5,goals_l5,assists_l5,cards_yellow_l5,game_count_l5,passes_l5_minus_szn,opp_passes_against
0,Y,1,0,90,0,0,0,0,0,2,...,0.000000,0.00,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,1.026736
1,Y,2,0,89,0,0,1,0,0,1,...,1.000000,27.00,0.000000,0.0,0.00,0.0,0.00,1.0,0.0,0.896010
2,Y,3,0,90,0,0,6,0,0,1,...,0.500000,26.50,0.500000,0.0,0.00,0.0,0.00,1.5,0.0,1.041747
3,Y,4,0,90,0,0,1,0,0,4,...,0.666667,28.00,0.666667,0.0,0.00,0.0,0.00,2.0,0.0,1.263205
4,Y,5,0,79,0,0,4,0,0,0,...,1.250000,32.75,0.500000,0.0,0.25,0.0,0.25,2.5,0.0,0.834888


In [ ]:
df.shape

#save df as a csv file
df.to_csv('output.csv', index=False)


Now we have a list of every game played by a Premier League player, as well as their season and last 5 totals going into the game.

From here, we can use this data to determine what is a better predictor of a player's result by calculating the correlation of both variables

In [ ]:
import statsmodels.api as sm

# Sample data
X = np.array(df['passes_szn'])
Y = np.array(df['passes'])

# Add a constant to the independent variable for the intercept
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()


# Calculate the residual standard deviation
print(model.summary())
print("Residual Standard Deviation:", np.sqrt(model.scale)) 


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.407
Method:                 Least Squares   F-statistic:                     1850.
Date:                Mon, 08 Jul 2024   Prob (F-statistic):          3.48e-308
Time:                        21:47:01   Log-Likelihood:                -11705.
No. Observations:                2698   AIC:                         2.341e+04
Df Residuals:                    2696   BIC:                         2.343e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.0290      0.663     12.108      0.0

In [ ]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.407
Method:                 Least Squares   F-statistic:                     1850.
Date:                Mon, 08 Jul 2024   Prob (F-statistic):          3.48e-308
Time:                        21:47:01   Log-Likelihood:                -11705.
No. Observations:                2698   AIC:                         2.341e+04
Df Residuals:                    2696   BIC:                         2.343e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.0290      0.663     12.108      0.000       6.729       9.329
x1             0.7583      0.018     43.009      0.000       0.724       0.793
==============================================================================
Omnibus:                      613.368   Durbin-Watson:                   1.766
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1878.072
Skew:                           1.152   Prob(JB):                         0.00
Kurtosis:                       6.377   Cond. No.                         69.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This shows a standard linear regression between a player's season to date total for passes attempted, and the resulting passes a player attempted in the game.
While the two variables are positively correlated (a r^2 value of 4/9 shows this, as well as a p value of zero), the high residual standard deviation shows that season averages alone isn't a great predictor of a player's single game output. 

Next, we will try adding a player's last 5 results to the regression, to see if this improves the accuracy of the model.

In [ ]:
X = np.array(df[['passes_szn','passes_l5']])
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.416
Method:                 Least Squares   F-statistic:                     961.6
Date:                Mon, 08 Jul 2024   Prob (F-statistic):          6.44e-316
Time:                        21:47:01   Log-Likelihood:                -11683.
No. Observations:                2698   AIC:                         2.337e+04
Df Residuals:                    2695   BIC:                         2.339e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.8839      0.658     11.977      0.000       6.593       9.175
x1             0.4114      0.055      7.456      0.000       0.303       0.520
x2             0.3563      0.054      6.629      0.000       0.251       0.462
==============================================================================
Omnibus:                      632.367   Durbin-Watson:                   1.856
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2008.461
Skew:                           1.174   Prob(JB):                         0.00
Kurtosis:                       6.515   Cond. No.                         98.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
print("Residual Standard Deviation:", np.sqrt(model.scale))

Residual Standard Deviation: 18.391383778693918


The improvement in the regression shows that using a player's last 5 games as data slightly improves the predictive power of the model. 

Note that, as season averages has a greater slope than last 5 averages, it can be concluded that season averages are a slightly better predictor of future outcomes than a player's last 5 averages, however the best way to predict future outcomes is to consider both averages. 

Next, let's try factoring in the opponent's averages against, and see if this improves the model more.  Personally, I think this is the second biggest factor in determining output, so I am expecting a large decrease in residential standard deviation. 

In [ ]:
X = np.array(df[['passes_szn','passes_l5_minus_szn', 'opp_passes_against']])
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()

print("Residual Standard Deviation:", np.sqrt(model.scale))
model.summary()

Residual Standard Deviation: 17.211905584405866


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.489
Model:                            OLS   Adj. R-squared:                  0.489
Method:                 Least Squares   F-statistic:                     859.6
Date:                Mon, 08 Jul 2024   Prob (F-statistic):               0.00
Time:                        21:47:01   Log-Likelihood:                -11504.
No. Observations:                2698   AIC:                         2.302e+04
Df Residuals:                    2694   BIC:                         2.304e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -34.8414      2.268    -15.360      0.000     -39.289     -30.393
x1             0.7760      0.016     47.230      0.000       0.744       0.808
x2             0.3683      0.050      7.319      0.000       0.270       0.467
x3            42.5596      2.175     19.571      0.000      38.295      46.824
==============================================================================
Omnibus:                      489.101   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1430.249
Skew:                           0.938   Prob(JB):                    2.66e-311
Kurtosis:                       6.033   Cond. No.                         354.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This provides an improvement than adding last 5 stats, however, there is still a large amount of deviance shown in the data, meaning more variables might be needed to predict it.  A good suggestion might be to show the expected standard deviation of a poisson process with means at the line of best fit, in order to provide a target standard deviation in which to aim for.